In [1]:
%matplotlib inline
import edward as ed
from edward.models import Normal
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# our functions
import setup
import data
import visualizations
import basis_functions
pd.set_option('float_format', '{:f}'.format)
setup.set_random_seeds(42)
plt.style.use("seaborn-talk")
sns.set_context("talk")

In [2]:
manhattan = data.get_borough_data("data/preprocessed.csv", "Manhattan")
ues_to_msh = data.get_neighborhood_to_neighborhood("Morningside Heights", "Upper East Side-Carnegie Hill", manhattan)

In [3]:
indicator_cols = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude", "manhattan_distance", "pickup_hour", "pickup_timestamp"]
y_cols = ["trip_duration"]

x_train_raw, y_train_raw, x_test_raw, y_test_raw = data.train_test_split(ues_to_msh, 0.1, indicator_cols, y_cols)
x_train = data.standardize_cols(x_train_raw)
x_test = data.standardize_cols(x_test_raw)


# Box's Loop - Iteration 3

## Model: Gaussian Process

Gaussian processes (GPs) are a supervised machine learning algorithm that measures the similarity between input data points, using a predefined kernel function, to predict the value of an unseen data point. We like to think about GPs as: defining an infinite distribution on functions over a continuous sapce where we observe some data points and want to assign probabilities to all the ways a line could be drawn through those points. The idea is that we hope these lines (or functions) we draw are similar with high probability to that true function which we will never know. 

Drawing a parallel to our polynomial basis GLM example: instead of choosing a polynomial basis function of degree four, with GPs using a kernel function we'd like to infer the true function from our data. Hence, GPs allow us retain the flexibility of capturing non-linearities in our data but accounting for "infinite" numbers of basis functions. 

### Interpretation of Kernel Function

GPs are parameterized by a pre-determined *kernel function* which is positive-semidefinite covariance matrix that calculated distanced between every pair of $N$ observed points. The Kernel function must be a square matrix and allows use to explore *smoothness* and *periodicity* in our observed data. 

In our analysis we will explore a few kernel functions but note there are many popular kernel functions, some more applicable than others and exploring them all was impossible. 

### Model Overview

A GP can be specified entirely by it's Kernel Function and mean (often assumed to 0): 

Where,

**Prior:** $p(f) = GP(\ 0,\ K(x,x',\theta)\ )$ where $\theta$ = *length_scale* $l_f$, sigma $\sigma_f$

**Likelihood:**  $p(y\ |\ f,\ X,\ \theta) = GP(\ f,\ K(x,x', \theta)\ )$

In [80]:
def gp_reg_invert_K(x, y, x_star, kernel):
    K = kernel(x)
    k_star = kernel(x_star, x)
    k_star_star = kernel(x_star, x_star)
    mu_n, sigma_n_sqr = tf.nn.moments(y, 1)
    K_noisy = K + sigma_n_sqr * tf.eye(tf.cast(x.get_shape()[0], tf.int32), dtype=tf.float64)
    K_noisy_inv = tf.matrix_inverse(K_noisy)
    f_bar = tf.matmul(tf.matmul(k_star, K_noisy_inv), y)
    tmp = tf.matmul(tf.matmul(k_star, K_noisy_inv), k_star, transpose_b=True)
    v = k_star_star - tmp
    return f_bar, v

y = tf.constant(y_train_raw.as_matrix())
x = tf.constant(x_train)
x_star = tf.constant(x_test.iloc[:2, :])

mu, var = tf.nn.moments(y, 1)

def my_cov(X, X2=None, lengthscale=1.0, variance=1.0):
    lengthscale = tf.convert_to_tensor(lengthscale, dtype=tf.float64)
    variance = tf.convert_to_tensor(variance, dtype=tf.float64)

    X = tf.convert_to_tensor(X)
    X = X / lengthscale
    Xs = tf.reduce_sum(tf.square(X), 1)
    if X2 is None:
        X2 = X
        X2s = Xs
    else:
        X2 = tf.convert_to_tensor(X2)
        X2 = X2 / lengthscale
        X2s = tf.reduce_sum(tf.square(X2), 1)

    square = tf.reshape(Xs, [-1, 1]) + tf.reshape(X2s, [1, -1]) - 2 * tf.matmul(X, X2, transpose_b=True)
    output = variance * tf.exp(-square / 2)
    return output

f, v = gp_reg_invert_K(x, y, x_star, my_cov)
mean = np.reshape(f.eval(session=tf.Session()), -1)
covariance = v.eval(session=tf.Session())
np.random.multivariate_normal(mean, covariance)

array([ 11.79125001,  18.39803253])

In [ ]:
    
# def gp_reg(X, y, kernel, sigma_sqr_n, x_star):
#     K = kernel(X)
#     noise = sigma_sqr_n * tf.eye(tf.cast(X.get_shape()[0], tf.int32), dtype=tf.float64)
#     K_noisy = K + noise
#     L = tf.cholesky(K_noisy)
#     z = tf.cholesky_solve(L, y)
#     alpha = tf.cholesky_solve(tf.matrix_transpose(L), z)
#     xs = tf.unstack(x_star)
#     fs, vs = [], []
#     k_star = kernel(X, x)
#     f = tf.matmul(k_star, alpha, transpose_a=True)
#     print(f.eval(session=tf.Session()))
#     f = 
#     for idx, x in enumerate(xs):
#         k_star = kernel(x, X)
#         print(k_star)
#         return
#         k_star_star = kernel(x_star, x_star)
#         fs.append(tf.matmul(k_star, alpha))
#         v = tf.cholesky_solve(L, tf.reshape(k_star[idx], [633, 1]))
#         print(v)
#         print(tf.matmul(v, v, transpose_a=True))
#         print(k_star_star)
#         vs.append(k_star_star - tf.mat)
        
